In [1]:
# imports necesarios
from sklearn.datasets import fetch_openml
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from sklearn.ensemble import HistGradientBoostingClassifier
import time

### MNIST con GradientBoosting
En esta segunda sesión práctica vamos a entrenar clasificadores basados en GradientBoosting.
Claramente estos clasificadores son más costosos de entrenar.


In [2]:
mnist = fetch_openml("mnist_784")

print(mnist.data.shape)
print(mnist.target.shape)

data = mnist.data
targets = mnist.target 

targets=targets.to_numpy()
targets=np.int8(targets)

data=data.to_numpy()
data=np.float32(data)

/usr/lib/python3.10/site-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


(70000, 784)
(70000,)


### Partición de los datos

Vamos a partir los datos en tres conjuntos: training, validation y test. Con un 80%, 10% y 10% respectivamente. 

Emplearemos el conjunto de training para aprender los parámetros del modelos, el conjunto de validation para escoger los mejores hiperparámetros. Finalmente reportaremos el resultado final sobre el conjunto de test.

### **Ejercicio 1**    

Realiza la partición de los datos en las particiones definidas (80%,10%,10%)

In [3]:
# Partición de los datos
tam_test = 0.1
tam_val = 0.1
tam_train = 0.8

X_train, X_test, y_train, y_test = train_test_split(data, targets, test_size=tam_test, random_state=23)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=tam_val/(tam_train + tam_test), random_state=23) 


### **Ejercicio 2**   

Prueba un clasificador GradientBoosting

In [ ]:
## GradientBoosting
clf = GradientBoostingClassifier().fit(X_train, y_train)
acc = accuracy_score(y_val, clf.predict(X_val))
print('La precisión de {0!s} es {1:.1%}'.format(clf, acc))


Podrás comprobar como es un clasificador mucho más lento. 

### **Ejercicio 3**   

Para reducir el coste computacional se propone crear un pipeline donde se reduzca el número de características mediante PCA. En concreto el número de componentes (dimensiones) a las que reducimos con PCA es un hyperparámetro que tendrás que estimar con el conjunto de validación.

In [ ]:
## Mejorar velocidad reduciendo dimensionalidad con PCA
for n_components in [2, 4, 8, 16, 32]:
        clf = make_pipeline(PCA(n_components=n_components) if n_components>0 else None,
                GradientBoostingClassifier()).fit(X_train, y_train)
        acc = accuracy_score(y_val, clf.predict(X_val))
        print('La precisión de {0!s} es {1:.1%}'.format(clf, acc))

De entre los diferentes parámeros que tiene el clasificador GradientBoosting de sklearn, cabría destacar:

**learning_rate**

**n_estimators**

**min_samples_split**

**max_features** 


Para más información leer la documentación en sklearn.

Alguno de estos parámetros influyen considerablemente en la velocidad de optimización. Por ejemplo **max_features** y **min_samples_split** entre otros.



### **Ejercicio 4**   

Se propone variar alguno de estos parámetros para ver si se obtiene una similar tasa de acierto pero con mejor velocidad.


In [ ]:
# Calcular tiempo y tasa de acierto de la versión original con PCA
ini = time.time()
clf = make_pipeline(PCA(n_components=16),
            GradientBoostingClassifier()).fit(X_train, y_train)
acc = accuracy_score(y_val, clf.predict(X_val))
fin = time.time()
print('La precisión de {0!s} es {1:.1%} y tarda {2:.5}'.format(clf, acc, fin - ini))

In [ ]:
# Calcular tiempo y tasa de acierto modificando alguno de los parámetros propuestos
ini = time.time()
clf = make_pipeline(PCA(n_components=16),
            GradientBoostingClassifier(max_features=20, n_stimators=100, min_samples_split=10)).fit(X_train, y_train)
acc = accuracy_score(y_val, clf.predict(X_val))
fin = time.time()
print('La precisión de {0!s} es {1:.1%} y tarda {2:.5}'.format(clf, acc, fin - ini))

Finalmente sklearn propone otro tipo de algoritmo de GradientBoosting que soporta paralelismo con OMP además de otras mejoras computacionales basadas en la discretización de las componentes mediante un histograma: el HistGradientBoostingClassifier. Su tiempo de ejecución es mucho mejor. Además se pueden obtener mejores resultados.



### **Ejercicio 5**   

Pruébalo y compara los tiempos de ejecución.

In [ ]:
# Probar HistGradientBoostingClassifier 
clf = make_pipeline(PCA(n_components=16),
            HistGradientBoostingClassifier()).fit(X_train, y_train)
acc = accuracy_score(y_val, clf.predict(X_val))
print('La precisión de {0!s} es {1:.1%}'.format(clf, acc))

### **Ejercicio 6**   

Prueba los parámetros del HistGradientBoostingClassifier que mejoren la tasa de acierto. En cualquier caso la selección de estos parámetros debe seguir el protocolo de experimentación antes expuesto. Esto es, escoger el mejor parámetro con datos de validación y reportar resultados con los datos de test.

In [ ]:
max = -1
bestN_comp = -1
for n_component in [8, 16, 32]:
    clf = make_pipeline(PCA(n_components=n_component),
                HistGradientBoostingClassifier()).fit(X_train, y_train)
    acc = accuracy_score(y_val, clf.predict(X_val))
    print('La precisión de {0!s} es {1:.1%}'.format(clf, acc))

    if acc > max:
        bestN_comp = n_component
        max = acc

In [7]:
bestN_comp=16
for md in [8, 16, 32]:
    for mB in [32, 64, 128]:
        ini = time.time()
        clf = make_pipeline(PCA(n_components=bestN_comp),
                    HistGradientBoostingClassifier(max_depth=md, max_bins=mB)).fit(X_train, y_train)
        acc = accuracy_score(y_val, clf.predict(X_val))
        fin = time.time()
        print('La precisión de {0!s} es {1:.1%} y tarda {2:.5}'.format(clf, acc, fin - ini))

La precisión de Pipeline(steps=[('pca', PCA(n_components=16)),
                ('histgradientboostingclassifier',
                 HistGradientBoostingClassifier(max_bins=32, max_depth=8))]) es 95.0% y tarda 31.902
La precisión de Pipeline(steps=[('pca', PCA(n_components=16)),
                ('histgradientboostingclassifier',
                 HistGradientBoostingClassifier(max_bins=64, max_depth=8))]) es 94.8% y tarda 31.771
La precisión de Pipeline(steps=[('pca', PCA(n_components=16)),
                ('histgradientboostingclassifier',
                 HistGradientBoostingClassifier(max_bins=128, max_depth=8))]) es 95.2% y tarda 31.609
La precisión de Pipeline(steps=[('pca', PCA(n_components=16)),
                ('histgradientboostingclassifier',
                 HistGradientBoostingClassifier(max_bins=32, max_depth=16))]) es 95.0% y tarda 31.998
La precisión de Pipeline(steps=[('pca', PCA(n_components=16)),
                ('histgradientboostingclassifier',
                 HistGra

In [ ]:
for minL in [2, 3, 4, 5]:
        clf = make_pipeline(PCA(n_components=32),
                        HistGradientBoostingClassifier(min_samples_leaf=minL))
        clf = clf.fit(X_train, y_train)
        y_pred = clf.predict(X_val)
        acc = accuracy_score(y_val, y_pred)
        print(clf, acc)

Pipeline(steps=[('pca', PCA(n_components=32)),
                ('histgradientboostingclassifier',
                 HistGradientBoostingClassifier(min_samples_leaf=2))]) 0.9604342236823311
Pipeline(steps=[('pca', PCA(n_components=32)),
                ('histgradientboostingclassifier',
                 HistGradientBoostingClassifier(min_samples_leaf=3))]) 0.9587201828310241
Pipeline(steps=[('pca', PCA(n_components=32)),
                ('histgradientboostingclassifier',
                 HistGradientBoostingClassifier(min_samples_leaf=4))]) 0.9628624482216827
Pipeline(steps=[('pca', PCA(n_components=32)),
                ('histgradientboostingclassifier',
                 HistGradientBoostingClassifier(min_samples_leaf=5))]) 0.9614340808455935